In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os

In [ ]:
#load
f = open('data.json')
games = json.load(f)

In [ ]:
more = True
initial = True
last = 0
games = []

while True:
  if initial:
    data = requests.get(f'https://api.steampowered.com/IStoreService/GetAppList/v1/?access_token={os.environ['TOKEN']}&include_games=true&max_results=50000')
    initial = False
  else:
    data = requests.get(f'https://api.steampowered.com/IStoreService/GetAppList/v1/?access_token={os.environ['TOKEN']}&include_games=true&last_appid={str(last)}&max_results=50000')
  data = data.json()['response']
  games += data['apps']
  if 'have_more_results' not in data:
    break
  if data['have_more_results'] == False:
    break
  last = data['last_appid']

In [ ]:
cookies = dict(birthtime="-2208959999")
for entry in games:
  appid = entry['appid']
  if appid <= last:
    continue
  page = requests.get(f'https://store.steampowered.com/app/{str(appid)}')
  soup = BeautifulSoup(page.content, "html.parser")
  tagclass = soup.find(class_="glance_tags popular_tags")
  if tagclass == None:
    entry["tags"] = []
    continue
  tags = [tag.get_text() for tag in tagclass.find_all("a")]
  for i in range(len(tags)):
    tags[i] = tags[i].replace("\r","")
    tags[i] = tags[i].replace("\n","")
    tags[i] = tags[i].replace("\t","")
  entry["tags"] = tags
  last = appid

In [ ]:
with open("data.json", "w") as outfile:
  json.dump(games, outfile)